In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import graphviz 
# might need to install -- need *both* python and system
# so use Anaconda (or pip) to install graphviz, and also https://graphviz.org/download/
# this was a non-trivial install on Mac (depends on Xcode command-line tools)
# `pip install graphviz` and `brew install graphviz`
import datetime as dt

%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error

In [2]:
bb = pd.read_csv("../../Data/1999_2021/aggragate_daily_data_2000to2020.csv", index_col = "date").loc['2008-01-01':'2017-12-31']

weather = pd.read_csv("../../Data/historic_weather_all_SP_daily.csv", index_col = "date")



## Change .fillna(0) to something more sophisticated?

In [3]:
# rain = weather.pivot(columns='station', values=['rain_mm']).fillna(0)[[('rain_mm',    'A701 SAO PAULO - MIRANTE'), ('rain_mm',               'A713 SOROCABA'), ('rain_mm',      'A744 BRAGANCA PAULISTA'),  ('rain_mm',                'A755 BARUERI'), ('rain_mm',             'A711 SAO CARLOS'), ('rain_mm',           'A741 BARRA BONITA')]]
rain = weather.pivot(columns='station', values=['rain_mm']).fillna(0)[[('rain_mm',    'A701 SAO PAULO - MIRANTE'), ('rain_mm',               'A713 SOROCABA'), ('rain_mm',             'A711 SAO CARLOS'), ('rain_mm',           'A741 BARRA BONITA')
]]

## Change lags here if you want

In [4]:
rain_agg = bb

for i in [1,3,7,10,30]: #all lags considered in the random forest
    rain_lag = rain.shift(i).rename(columns={'rain_mm':'rain_mm_' + str(i)})
    rain_agg = rain_agg.join(rain_lag).dropna()

/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pa

In [5]:
# rain_agg.corr()

In [6]:
X_agg = rain_agg.drop('maximum_demand', axis = 1)
y_agg = rain_agg.maximum_demand
X_agg_train, X_agg_test, y_agg_train, y_agg_test = train_test_split(X_agg, y_agg, train_size = 0.8, random_state = 0)

In [7]:
random_forest_rain_agg = RandomForestRegressor(max_features = 3, random_state = 17)

random_forest_rain_agg.fit(X_agg_train, y_agg_train)

pred_agg = random_forest_rain_agg.predict(X_agg_test)

Importance_agg = pd.DataFrame({'Importance':random_forest_rain_agg.feature_importances_*100}, 
                          index = X_agg.columns)

Importance_agg = Importance_agg.sort_values(by = 'Importance', axis = 0)

print("random forest MAE:", mean_absolute_error(y_agg_test, pred_agg))

random forest MAE: 927.3292053625172


In [8]:
Importance_agg = pd.DataFrame({'Importance':random_forest_rain_agg.feature_importances_*100}, 
                          index = X_agg.columns)

Importance_agg = Importance_agg.sort_values(by = 'Importance', axis = 0)

In [9]:
Importance_agg.sort_values('Importance',ascending=False).head(30)

,Importance
energy_charge,33.589295
Unnamed: 0,18.361616
energy_stored,6.292149
water_level,3.657074
volume_used,3.620037
energy_generated,3.293212
effluent_flow,3.252343
influent_flow,3.044568
poured_flow,2.539544
"(rain_mm_3, A701 SAO PAULO - MIRANTE)",1.318635
